## Better!

In this notebook I will try refactoring the code to use the api more efficiently.


##### Load Libraries


In [200]:
import os
import json
import base64
from typing import List
from openai import OpenAI
from dotenv import load_dotenv
from colorama import Back, Style, Fore


##### Load Environment Variables


In [201]:
load_dotenv()
api_key = os.environ["OPENAI_API_KEY"]

##### Data File Access


In [202]:
PATH_TO_JSON_FILE = "../data/visa_information.json"

try:
    with open(PATH_TO_JSON_FILE) as file:
        visa_information = json.load(file)
except:
    raise Exception("Failed to load json file.")

##### Tools and System Prompts


In [203]:
tools: [
    {
        "type": "function",
        "function": {
            "name": "get_visa_information",
            "description": """Retrieve visa information between two countries. 
                            🚨 This function must be called immediately without explaining what you are doing. 
                            🚨 DO NOT generate visa details from prior knowledge. Only return function response.
                            Do not pass full country names—only ISO 3166-1 alpha-3 codes are accepted.
                            Example: 'origin_country': 'PRT', 'destination_country': 'FRA'.""",
            "parameters": {
                "type": "object",
                "properties": {
                    "origin_country": {
                        "type": "string",
                        "description": "ISO 3166-1 alpha-3 code of the passport country (e.g., 'PRT' for Portugal, 'USA' for United States)."
                    },
                    "destination_country": {
                        "type": "string",
                        "description": "ISO 3166-1 alpha-3 code of the destination country (e.g., 'FRA' for France, 'JPN' for Japan)."
                    }
                },
                "required": ["origin_country", "destination_country"],
                "additionalProperties": False
            },
            "strict": True
        }
    }
]

SYSTEM_PROMPT = {
            "role": "system",
            "content": 
            """
            You are an AI assistant that provides visa information by calling a function. Your role is to:
            
            1. Ask the user for their passport country and destination country **by name** (e.g., 'Portugal' instead of 'PRT').
            2. For the origin country, the user may provide:
                - The country name as text.
                - A passport image file (acceptable formats: .png, .jpg).
            3. Before calling the function, **always convert country names to ISO 3166-1 alpha-3 codes** (e.g., 'Portugal' → 'PRT', 'France' → 'FRA').
            4. 🚨 Call the function **immediately** without stating that you are going to call it.
            5. Use **only** the function response to provide visa details. 🚨 **Do not use any prior knowledge or generate visa details yourself.**
            6. The function must be called with **only** ISO 3166-1 alpha-3 codes. Do not pass full country names.
            7. If the model mistakenly generates a country name instead of an alpha-3 code, immediately correct it and retry.
            8. Do not generate visa information yourself—always rely on the function response.
            9. Do not explain what you are doing—just return the visa information after calling the function.
            10. Ensure responses are clear, concise, and professional. Ask clarifying questions before making the function call if necessary.
            🚨 IMPORTANT: Never say 'Fetching visa information' or 'Calling the function'. Instead, simply call the function and return the response.
            🚨 IMPORTANT: Never use your own knowledge about visa requirements. Always rely on the function response.
            """
        }


##### Adjust Data Structure For Easier Lookups


In [204]:
new_visa_information = {}

for origin, visa_info in visa_information.items():
    new_visa_information[origin] = {}
    """ 
        {
            destination: {}
        }
    """

    for destination, details in visa_info.get("visaFree", {}).items():
        new_visa_information[origin][destination] = {
            "type": "visaFree",
            "maxStay": details["maxStay"]
        }
    
    for destination, details in visa_info.get("visaOnArrival", {}).items():
        new_visa_information[origin][destination] = {
            "type": "visaOnArrival",
            "maxStay": details["maxStay"]
        }
    
    """
    {
        destination: 
        {
            origin: 
            {
                type: string,
                "maxStay": number    
            }    
        }
    }        
    """

##### Function To Get Visa Information

For the sake of scalability, we will use a dictionary with the name of the function and a reference to the function in the code, this way we can add functions easily to our system by appending to this dictionary.


In [205]:
async def get_visa_information(origin_country: str, destination_country: str, data = new_visa_information):    
    if not origin_country and not destination_country:
        raise ValueError("Both 'origin' and 'destination' countries are required.")
    
    # Check if destination country exists in the data
    if destination_country not in data:
        return "No visa information available for destination country"
    
    # Get listed countries for the destination country
    destination_data = data[destination_country]
    
    # If origin country isn't found within destination country, default to 'visa required'
    visa_info = destination_data.get(origin_country, {
        'type': 'visa required',
        'maxStay': "None"
    })
    
    return {
        'origin country': origin_country,
        'destination country': destination_country,
        'visa type': visa_info['type'],
        'maxStay': visa_info['maxStay']
    }

##### Utility Functions


In [211]:
from pathlib import Path

def encode_image(image_name):
    path_to_image = Path().cwd().parent / "data" / "passports" / image_name

    with open(path_to_image, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

# https://platform.openai.com/docs/guides/vision/quick-start#uploading-base64-encoded-images
def create_image_prompt(image_name):
    base64_image = encode_image(image_name)

    return {
            "role": "user",
            "content": [
                {"type": "text", "text": "Here is a picture of a passport. Extract the issuer country and return the ISO Alpha-3 code of the corresponding country."},
                {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"} },
            ]
        }
        


In [207]:
# Add More Functions Here
function_calls_dict = {
    "get_visa_information": get_visa_information,
}

##### Function Calling Service

This service class will be responsible for:

- Access to the functions dictionary.
- Extract tool calls from the assistant messages.
- Loop through the tool calls and call each function respectively.
- Append each tool call and its subsequent result into an array to be appended eventually to the main message stack.


In [208]:
class FunctionCallService():
    def __init__(self, functions):
        self.functions = functions
        self.tool_call_messages = []
        
    async def run(self, tool_call_prompt):
        self.tool_call_messages = []
        
        for tool_call in tool_call_prompt.tool_calls:
            # Extract Function Name and Arguments
            name = tool_call.function.name
            args = json.loads(tool_call.function.arguments)

            # Call The Function
            result = await self.__call_function(name, args)

            # In case our function returns an object
            if isinstance(result, object):
                result = json.dumps(result)
            
            self.tool_call_messages.append({
                "role": "tool",
                "tool_call_id":tool_call.id,
                "content": result
            })
            
        return self.tool_call_messages
            
    async def __call_function(self, name, args):
        if name in self.functions:
            func = self.functions[name]
            try:
                return await func(**args)
            except TypeError as e:
                return f"Error calling function '{name}': {str(e)}"
        else:
            return f"Error: Function '{name}' not found."
        
    def reset(self):
        self.tool_call_messages = []

##### Chat Manager

The main idea behind having a manager for our chat is to allow easier management for the messages stack.
In this class we mainly run completions, and those completions – based on their type (tool calls, message etc..) – we route the flow of the messages.


In [209]:
from typing import List
from openai import OpenAI

class ChatManager():
    def __init__(self, tools, api_key: str ,model: str = "gpt-4o-mini"):
        self.model = model
        self.tools = tools
        self.messages = [SYSTEM_PROMPT]
        self.client = OpenAI(api_key=api_key)
        self.functionCallService = FunctionCallService(function_calls_dict)
    
    async def next(self, message = None):
                
        if message is not None:
            self.messages.append(message)
        
        completion, is_function_calling = await self.__get_completion()
        
        # If there are function calling
        if is_function_calling:
            # Get the responses from the tools (Function Calls)
            tool_calls_responses = await self.functionCallService.run(completion)
            
            # Append the responses to the Messages Stack
            self.messages = self.messages + tool_calls_responses
            
            # Run a new completion with the new information
            completion, _ = await self.__get_completion()
            
        # Return the content of the prompt
        return completion.content
    
    async def __get_completion(self):
        response = self.client.chat.completions.create(
            model = self.model,
            tools = self.tools,
            messages = self.messages,
            tool_choice = "auto"
        )
        
        # Extract Message
        message = response.choices[0].message
        
        # Check if there is a function call
        is_function_calling = (
            hasattr(response.choices[0].message, "tool_calls") 
            and 
            message.tool_calls is not None)
        
        if not is_function_calling:
            self.messages.append({"role": "assistant", "content": message.content})
        else:
            self.messages.append(message)
            
        return message, is_function_calling
    
    def reset(self):
        self.messages = []
        self.functionCallService.reset()
    

### Prompting


In [ ]:
# Create Manager
manager = ChatManager(tools=tools, api_key=api_key)

display_messages = []

print("""Chat started. 
      \nTo provide passport picture, please include only the filename with its extension. 
      \nType 'quit' to exit
      \nType 'reset' to restart chat.
      """)

while True:
    
    raw_user_prompt = input()
    
    # Validation and Sanitization
    if raw_user_prompt == "":
        print(Fore.YELLOW + "System: Please provide a proper input.")
        continue
    
    # Quit Prompting
    if raw_user_prompt.lower() == "quit":
        print(Fore.RED + "Ending chat...")
        break
    
    # Reset Chat
    if raw_user_prompt.lower() == "reset":
        print(Fore.BLUE + "System: Resetting Chat..." + Style.RESET_ALL)
        manager.reset()
        display_messages = []
        continue
    
    print(Fore.CYAN + f"\nYou: {raw_user_prompt}")
    
    user_prompt = {"role": "user","content": raw_user_prompt}
    
    if raw_user_prompt.endswith(".png") or raw_user_prompt.endswith(".jpg"):
        user_prompt = create_image_prompt(raw_user_prompt)
    
    display_messages.append(raw_user_prompt)
    
    completion = await manager.next(user_prompt)
    
    display_messages.append({"role": "assistant", "content": completion})
    
    print(Fore.GREEN + f"\nAssistant: {completion}" + Style.RESET_ALL)